# CODE-DE/EO-Lab tutorial

<div style="text-align: right"><i> Beginner </i></div>

***
<center><h1> Prozessierung einer Vegetations-Zeitreihe mit FORCE Daten </h1></center>

***
**General Note 1**: Ausführung der Zellen durch pressen des <button class="btn btn-default btn-xs"><i class="icon-play fa fa-play"></i></button> button vom top MENU (oder `Shift` + `Enter`).<br>
<br>
**General Note 2**: Falls der Kern nich mehr arbeitet, im the top MENU, klicke <button class="btn btn-default btn-xs"><i class="fa fa-repeat icon-repeat"></i></button> button. Dann, im top MENU, clicke  "Run" aund wähle "Run All Above Selected Cell".<br>

**General Note 3**: Schauen Sie sich im [**FORUM**](https://forum.code-de.org/de/) um oder Kontaktieren Sie den Support ! <br>

***
Voraussetzung zur Nutzung dieses Notebooks ist eine VM auf CODE-DE oder EO-Lab.  
***

In diesem Tutorial möchten wir den FORCE Datacube verwenden, ohne die FORCE-Umgebung selbst zu nutzen.
Der FORCE-Datenwürfel wird hier ausführlich beschrieben:
https://force-eo.readthedocs.io/en/latest/howto/datacube.html
In der Filestruktur liegen die mit FORCE atmospärenkorrigten als BOA (Bottom of Atmosphere) Prodult bereits vor - ebenso die Wolkenmasken.

Die Daten mit dem Datenwürfel werden hier näher beschrieben:
https://force-eo.readthedocs.io/en/latest/components/lower-level/level2/format.html

Um die Daten in Python prozessieren zu können, benötigen wir einige standard Python Bibliotheken, die in der folgenden Zelle importiert werden.

In [ ]:
import rasterio
import glob
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from rasterio import plot
from osgeo import osr
import os
#somit werden alle Plots im Notebook auf diese Größe definiert. 
plt.rcParams['figure.figsize'] = [12,12]

FORCE ist in einem eigenen Daten-System organisiert: in 30 km x 30 km Rasterdatenblöcken. Diese sogenannten "Tiles" sind in einer wohldefinierten Proketion: https://epsg.io/3035 abgelegt.
Um zu wissen in welchem Tile wir uns befinden benötigen wir eine kleine Funktion, die uns das korrekte Tile ausrechnet (relativ zu einem bekannten Tile: hier X0044_Y0052).

Die beiden Variablen in der folgenden Zelle myX und myY rechnen die Dezimalkoordinaten unseres Standortes um und errechnen "unser" FORCE Tile.
Jahr und Monat müssen ebenfalls angegeben werden. Damit ist das Wo und Wann definiert.

In [ ]:
def reprojectPoint(EPSGin,EPSGout,xCoord,yCoord):
  src=osr.SpatialReference()
  tgt=osr.SpatialReference()
  src.ImportFromEPSG(EPSGin)
  tgt.ImportFromEPSG(EPSGout)
  transform = osr.CoordinateTransformation(src,tgt)
  coords = transform.TransformPoint(xCoord,yCoord)
  return coords[0:2]

# Koordinaten Stadt Bonn: 7.1022/50.7258; Stadt Essen: 7.0116/51.4556
myX = 7.0116
myY = 51.4556

#hier bitte angeben für welches Jahr und Monat gerechnet werden soll:
year='2020'
month='07'

tileSize = 30000
x1,y1=reprojectPoint(4326,3035,2.3852,49.992) # upper let coordinate of X0044 und Y0052
x2,y2=reprojectPoint(4326,3035,myX,myY) #Umprojetion von EPSG:4326 nach EPSG:3035

deltaX= round((x2-x1) / tileSize)
deltaY= round((y2-y1) /tileSize)
tileX='X00'+str(44+deltaX)
tileY='Y00'+str(52-deltaY)
tile=tileX+'_'+tileY

Nun erfolgt eine Datensuche direkt auf dem Filesystem und alle Sentinel-2 BOA Daten eines bestimmten Monats werden gesucht. "SEN2" steht hier für Sentinel-2.

In [ ]:
path='/codede/community/FORCE/C1/L2/ard/'+tile+'/' #Datenpfad der FORCE Kollektion 
sen2_files = "*SEN2?_BOA.tif"
globSearch = path+year+month+sen2_files
#print(globSearch)
files=glob.glob(globSearch)
print('Es wurden ' + str(len(files)) + ' Szenen im Tile ' + tile +' gefunden:')
print(files)


Für die Datenprozessierung benötigen wir eine Wolkenmaske, so dass bewölkte Pixel ignoriert werden können.

In [ ]:
def maskCloud(inFile):
  f=inFile.replace('BOA','QAI')
  src=rasterio.open(f)
  out_meta = src.meta
  QA=src.read() << 12
  mask = np.where(((QA > 2**12) ),1,0)
  
  return mask

Bei Fragen zur Wolkenmaskierung in FORCE könnte dieses Tool hilfreich sein: 
https://bit-flag-renderer.readthedocs.io/en/latest/index.html

Nun kommt der Kern des Programms. Wir iteriern über alle Sentinel-2 Szenen, berechnen jeweils den EVI (oder wahlweise NDVI), maskieren die Wolken und summieren die wolkenfreien Pixel. Am Ende erfolgt die Division durch die Anzahl der wolkenfreien Pixel zur Mittelwertbildung.

In [ ]:
#öffne ein Band um die Metadaten zu lesen
src=rasterio.open(files[0])
band8=src.read([8])
out_meta = src.meta
src.close() # schliesse das Band wieder
#wir legen leere numpy arrays der gleichen Dimensionen an
vi_sum = np.zeros_like(band8, dtype=np.float32)
vi_count = np.zeros_like(band8, dtype=np.uint8)

count=0
for f in files: # Schleife, die über alle Dateien iteriert.
    src=rasterio.open(f)
    NIR=src.read([8])/10000.0 # Skalierung in Force: Reflektanz * 10000
    RED=src.read([4])/10000.0
    BLUE = src.read([2])/10000.0
    src.close() 
    
    imgDate=os.path.basename(f)[0:8]         
    EVI = 2.5*(NIR-RED)/((NIR+6*RED-7.5*BLUE)+1) 
    #Alternativ mit NDVI die beiden folgden Zeilen auskommentieren.. 
    #NDVI = (NIR-RED)/(NIR+RED)
    #EVI = NDVI
    EVI[(EVI >1) | (EVI < -1)] = np.nan
    mask = np.zeros_like(NIR, dtype=np.uint8)
    clouds = maskCloud(f) 
    #nodata value in FORCE=-9999  
    mask = np.where((NIR == -9999) | np.isnan(EVI) | (clouds == 1), np.nan, 1)
    # Addition des EVI innerhalb der Schleife               
    vi_sum += np.where(np.isnan(mask), 0, EVI)
    vi_count += np.where(np.isnan(mask), 0, 1).astype(np.uint8)
    count += 1
    print("Processing image: ",count)
    src.close()
    
#Summe dividiert durch Anzahl ergibt den Mittelwert. Hier könnte z.B. der Median hilfreich sein.            
vi_average = vi_sum / vi_count

Zur Visualisuertung hier im Notebook nutzen wir eine simple Plot Funktion in pyplot: https://matplotlib.org/stable/tutorials/pyplot.html

In [ ]:
# RGB des letzen Bildes in der Liste
%matplotlib inline
RGB=np.stack([BLUE,NIR,RED],axis=-1).squeeze() 
RGB[(RGB > 1)] = 1
RGB[(RGB < 0)] = 0
plt.imshow(RGB)
plt.title(imgDate + ' - RGB: BLUE, NIR, RED')

Der gemittelte EVI über einen Monat als gemitteltes Bild-Komposit aller Aufnahmen im Monat.

In [ ]:
plt.imshow(np.squeeze(vi_average),cmap='jet',vmin=0,vmax=0.8)
plt.title('EVI')
plt.colorbar()

Die Anzahl der "guten" Pixel im Bild-Komposit als Resultat der Wolkenmaskierung.


In [ ]:
plt.imshow(np.squeeze(vi_count),cmap='inferno',vmin=0,vmax=count)
plt.title('Anzahl verwendeter Pixel')
plt.colorbar()

EVI und die Anzahl der "guten" Pixel werden nun als zwei seperate Dateien in das Verzeichnis /home/eouser/ geschrieben und können z.B. mit QGIS angesehen werden.

In [ ]:
kwargs = out_meta
kwargs.update(
    dtype=rasterio.float32,count=1,
    compress='lzw')
with rasterio.open('/home/eouser/'+year+month+'_evi.tif', 'w',**kwargs) as dst:
        dst.write(vi_average.astype(rasterio.float32))
        
#write count image 
kwargs.update(
    dtype=rasterio.uint8,count=1,nodata=255)
with rasterio.open('/home/eouser/'+year+month+'_count.tif', 'w',**kwargs) as dst:
        dst.write(vi_count.astype(rasterio.uint8))